# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.api as sm

In [4]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

## Carregando o data frame e removendo as colunas das quais não serão interessantes para esse modelo

In [6]:
df_encoded = df.drop(columns=['Unnamed: 0', 'id_cliente', 'data_ref']).copy()
df_encoded.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


## Determinando quais valores mais ocorrem em cada categoraia qualitativa para posterior analise.

In [7]:
for i in df_encoded.columns:
    print(f'{i}: ({df_encoded[i].value_counts().index[0]}: {df_encoded[i].value_counts().iloc[0]}) \n')

sexo: (F: 10119) 

posse_de_veiculo: (False: 9140) 

posse_de_imovel: (True: 10143) 

qtd_filhos: (0: 10376) 

tipo_renda: (Assalariado: 7633) 

educacao: (Secundário: 8895) 

estado_civil: (Casado: 10534) 

tipo_residencia: (Casa: 13532) 

idade: (40: 538) 

tempo_emprego: (4.216438356164384: 38) 

qt_pessoas_residencia: (2.0: 8181) 

renda: (728.96: 6) 



## Criando o primeiro modelo como todas as categorias e tratando as categorias qualitativas com base nos valores mais frequentes de cada categoria

In [8]:
formula = "np.log(renda) ~ C(sexo, Treatment('F')) + C(posse_de_veiculo, Treatment(False)) + C(posse_de_imovel, Treatment(True)) + qtd_filhos + C(tipo_renda, Treatment('Assalariado')) + C(educacao, Treatment('Secundário')) + C(estado_civil, Treatment('Casado')) + C(tipo_residencia, Treatment('Casa')) + idade + tempo_emprego + qt_pessoas_residencia"
reg = smf.ols(formula = formula, data = df_encoded).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Tue, 02 Sep 2025   Prob (F-statistic):               0.00
Time:                        10:59:30   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       6.6093      0.219     30.214      0.000       6.181       7.038
C(sexo, Treatment('F'))[T.M]                                    0.7874      0.015     53.723      0.000       0.759       0.816
C(posse_de_veiculo, Treatment(False))[T.True]                   0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel, Treatment(True))[T.False]                   -0.0829      0.014     -5.926      0.000      -0.110      -0.055
C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]             0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment('Assalariado'))[T.Empresário]           0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]         -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment('Assalariado'))[T.Servidor público]     0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment('Secundário'))[T.Primário]                0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment('Secundário'))[T.Pós graduação]           0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment('Secundário'))[T.Superior completo]       0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment('Secundário'))[T.Superior incompleto]    -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil, Treatment('Casado'))[T.Separado]                0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment('Casado'))[T.Solteiro]                0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment('Casado'))[T.União]                  -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment('Casado'))[T.Viúvo]                   0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment('Casa'))[T.Aluguel]                0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment('Casa'))[T.Com os pais]            0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment('Casa'))[T.Comunitário]           -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment('Casa'))[T.Estúdio]                0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment('Casa'))[T.Governamental]         -0.0129      0.039     -0.331      0.741      -0.089       0.064
qtd_filhos 

## Analisando quais categorias possuem um P>|t| maior ou igual a 5%

In [9]:
reg.summary().tables[1].data[1][4]
for i in range(len(reg.summary().tables[1])):
    try:
        if float(reg.summary().tables[1].data[i][4]) >= 0.05:
            print(f'{reg.summary().tables[1].data[i][4]}: {reg.summary().tables[1].data[i][0]}')
        else:
            continue
    except:
        continue

 0.360: C(tipo_renda, Treatment('Assalariado'))[T.Bolsista]
 0.201: C(tipo_renda, Treatment('Assalariado'))[T.Pensionista]
 0.844: C(educacao, Treatment('Secundário'))[T.Primário]
 0.394: C(educacao, Treatment('Secundário'))[T.Pós graduação]
 0.368: C(educacao, Treatment('Secundário'))[T.Superior incompleto]
 0.174: C(estado_civil, Treatment('Casado'))[T.União]
 0.415: C(tipo_residencia, Treatment('Casa'))[T.Aluguel]
 0.532: C(tipo_residencia, Treatment('Casa'))[T.Com os pais]
 0.407: C(tipo_residencia, Treatment('Casa'))[T.Comunitário]
 0.187: C(tipo_residencia, Treatment('Casa'))[T.Estúdio]
 0.741: C(tipo_residencia, Treatment('Casa'))[T.Governamental]


É possivel ver que a remoção dessas categorias trás pontos positivos para o modelo, apesar do R2 se tornar menor, mas isso nos trás um modelo mais robusto e com uma menor chance de overfitting.

## Gerando um novo modelo, agora removendo as categorias estatisticamente menos relevantes e fazendo nova analize

In [10]:
reg = smf.ols(formula = "np.log(renda) ~ C(sexo, Treatment('F')) + C(posse_de_veiculo, Treatment(False)) + C(posse_de_imovel, Treatment(True)) + qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia", data = df_encoded).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     940.8
Date:                Tue, 02 Sep 2025   Prob (F-statistic):               0.00
Time:                        10:59:31   Log-Likelihood:                -13672.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12419   BIC:                         2.742e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                         7.3243      0.044    167.889      0.000       7.239       7.410
C(sexo, Treatment('F'))[T.M]                      0.7694      0.015     52.676      0.000       0.741       0.798
C(posse_de_veiculo, Treatment(False))[T.True]     0.0569      0.014      4.022      0.000       0.029       0.085
C(posse_de_imovel, Treatment(True))[T.False]     -0.0866      0.014     -6.275      0.000      -0.114      -0.060
qtd_filhos                                        0.0338      0.019      1.735      0.083      -0.004       0.072
idade                                             0.0049      0.001      6.408      0.000       0.003       0.006
tempo_emprego                                     0.0610      0.001     59.075      0.000       0.059       0.063
qt_pessoas_residencia                            -0.0092      0.016     -0.566      0.572      -0.041       0.023
==============================================================================
Omnibus:                        1.240   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.538   Jarque-Bera (JB):                1.213
Skew:                           0.022   Prob(JB):                        0.545
Kurtosis:                       3.019   Cond. No.                         301.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Mais uma vez, determinando as novas categorias menos relevantes ao modelo seguindo a mesma regra dos 5%

In [11]:
reg.summary().tables[1].data[1][4]
for i in range(len(reg.summary().tables[1])):
    try:
        if float(reg.summary().tables[1].data[i][4]) >= 0.05:
            print(f'{reg.summary().tables[1].data[i][4]}: {reg.summary().tables[1].data[i][0]}')
        else:
            continue
    except:
        continue

 0.083: qtd_filhos
 0.572: qt_pessoas_residencia


## E agora, apos todos os tratamentos, analisando o novo modelo para as categorias que são relevantes para o modelo

In [12]:
reg = smf.ols(formula = "np.log(renda) ~ C(sexo, Treatment('F')) + C(posse_de_veiculo, Treatment(False)) + C(posse_de_imovel, Treatment(True)) + idade + tempo_emprego", data = df_encoded).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1315.
Date:                Tue, 02 Sep 2025   Prob (F-statistic):               0.00
Time:                        10:59:32   Log-Likelihood:                -13676.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12421   BIC:                         2.741e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                         7.3430      0.032    230.088      0.000       7.280       7.406
C(sexo, Treatment('F'))[T.M]                      0.7700      0.015     52.851      0.000       0.741       0.799
C(posse_de_veiculo, Treatment(False))[T.True]     0.0579      0.014      4.116      0.000       0.030       0.085
C(posse_de_imovel, Treatment(True))[T.False]     -0.0880      0.014     -6.379      0.000      -0.115      -0.061
idade                                             0.0044      0.001      5.871      0.000       0.003       0.006
tempo_emprego                                     0.0611      0.001     59.199      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.248   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.536   Jarque-Bera (JB):                1.221
Skew:                           0.022   Prob(JB):                        0.543
Kurtosis:                       3.019   Cond. No.                         211.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Conclusao:
    Apesar do modelo ter um R2 menos, como visto anteriormente, esse modelo é mais robusto e trás um resultado melhor por diminuir a chance de overfitting, tornando esse modelo mais proximo da realidade.